Load Julia packages (libraries) needed  for the snippets in chapter 0

In [1]:
using StanModels

CmdStan uses a tmp directory to store the output of cmdstan

In [2]:
ProjDir = rel_path_s("..", "scripts", "04")
cd(ProjDir)

### snippet 4.7

In [3]:
howell1 = CSV.read(rel_path("..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

Use only adults

In [4]:
df2 = filter(row -> row[:age] >= 18, df)
mean_weight = mean(df2[:weight])
df2[:weight_c] = convert(Vector{Float64}, df2[:weight]) .- mean_weight ;

Define the Stan language model

In [5]:
weightsmodel = "
data {
 int < lower = 1 > N; // Sample size
 vector[N] height; // Predictor
 vector[N] weight; // Outcome
}

parameters {
 real alpha; // Intercept
 real beta; // Slope (regression coefficients)
 real < lower = 0 > sigma; // Error SD
}

model {
 height ~ normal(alpha + weight * beta , sigma);
}

generated quantities {
}
";

Define the Stanmodel and set the output format to :mcmcchains.

In [6]:
stanmodel = Stanmodel(name="weights", monitors = ["alpha", "beta", "sigma"],model=weightsmodel,
  output_format=:mcmcchains);
# Input data for cmdstan
heightsdata = Dict("N" => length(df2[:height]), "height" => df2[:height], "weight" => df2[:weight_c]);

Sample using cmdstan

In [7]:
rc, chn, cnames = stan(stanmodel, heightsdata, ProjDir, diagnostics=false,
  summary=false, CmdStanDir=CMDSTAN_HOME);
# Describe the draws
describe(chn)

2-element Array{ChainDataFrame,1}

Summary Statistics

│ Row │ parameters │ mean     │ std      │ naive_se   │ mcse       │ ess    │
│     │ Symbol     │ Float64  │ Float64  │ Float64    │ Float64    │ Any    │
├─────┼────────────┼──────────┼──────────┼────────────┼────────────┼────────┤
│ 1   │ alpha      │ 154.598  │ 0.270395 │ 0.00427532 │ 0.00420428 │ 4000.0 │
│ 2   │ beta       │ 0.904176 │ 0.042255 │ 0.00066811 │ 0.00068671 │ 4000.0 │
│ 3   │ sigma      │ 5.10532  │ 0.194223 │ 0.00307094 │ 0.00291702 │ 4000.0 │

Quantiles

│ Row │ parameters │ 2.5%    │ 25.0%    │ 50.0%    │ 75.0%    │ 97.5%    │
│     │ Symbol     │ Float64 │ Float64  │ Float64  │ Float64  │ Float64  │
├─────┼────────────┼─────────┼──────────┼──────────┼──────────┼──────────┤
│ 1   │ alpha      │ 154.065 │ 154.422  │ 154.602  │ 154.784  │ 155.118  │
│ 2   │ beta       │ 0.81985 │ 0.877952 │ 0.903907 │ 0.930858 │ 0.991329 │
│ 3   │ sigma      │ 4.7386  │ 4.97408  │ 5.10037  │ 5.22952  │ 5.50707  │


Save the chains in a JLS file

In [8]:
serialize("m4.4s.jls", chn)

chn2 = deserialize("m4.4s.jls")

Object of type Chains, with data of type 1000×3×4 Array{Float64,3}

Iterations        = 1:1000
Thinning interval = 1
Chains            = 1, 2, 3, 4
Samples per chain = 1000
internals         = 
parameters        = alpha, beta, sigma

Summary Statistics

│ Row │ parameters │ mean     │ std      │ naive_se   │ mcse       │ ess    │
│     │ Symbol     │ Float64  │ Float64  │ Float64    │ Float64    │ Any    │
├─────┼────────────┼──────────┼──────────┼────────────┼────────────┼────────┤
│ 1   │ alpha      │ 154.598  │ 0.270395 │ 0.00427532 │ 0.00420428 │ 4000.0 │
│ 2   │ beta       │ 0.904176 │ 0.042255 │ 0.00066811 │ 0.00068671 │ 4000.0 │
│ 3   │ sigma      │ 5.10532  │ 0.194223 │ 0.00307094 │ 0.00291702 │ 4000.0 │

Should be identical to earlier result

In [9]:
describe(chn2)

2-element Array{ChainDataFrame,1}

Summary Statistics

│ Row │ parameters │ mean     │ std      │ naive_se   │ mcse       │ ess    │
│     │ Symbol     │ Float64  │ Float64  │ Float64    │ Float64    │ Any    │
├─────┼────────────┼──────────┼──────────┼────────────┼────────────┼────────┤
│ 1   │ alpha      │ 154.598  │ 0.270395 │ 0.00427532 │ 0.00420428 │ 4000.0 │
│ 2   │ beta       │ 0.904176 │ 0.042255 │ 0.00066811 │ 0.00068671 │ 4000.0 │
│ 3   │ sigma      │ 5.10532  │ 0.194223 │ 0.00307094 │ 0.00291702 │ 4000.0 │

Quantiles

│ Row │ parameters │ 2.5%    │ 25.0%    │ 50.0%    │ 75.0%    │ 97.5%    │
│     │ Symbol     │ Float64 │ Float64  │ Float64  │ Float64  │ Float64  │
├─────┼────────────┼─────────┼──────────┼──────────┼──────────┼──────────┤
│ 1   │ alpha      │ 154.065 │ 154.422  │ 154.602  │ 154.784  │ 155.118  │
│ 2   │ beta       │ 0.81985 │ 0.877952 │ 0.903907 │ 0.930858 │ 0.991329 │
│ 3   │ sigma      │ 4.7386  │ 4.97408  │ 5.10037  │ 5.22952  │ 5.50707  │


End of `m4.4s.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*